In [1524]:
import requests as r
import pandas as p
import json as j
__author__ = "Luis Oliveros"
__date__ = "8/29/2022"
__email__ = "luisqaz222@gmail.com"
__status__ = "Complete"

In [1525]:
def create_panthers_table(season, persons):
    """
    function that queries the NHL API /teams and /people endpoints for 
    all Florida Panthers players in respective season.

    :param season: the NHL season in format '20142015'
    :param persons: the number of Panthers players for the season 
    :return: a pandas dataframe.
    """ 
    #requests to NHL API to obtain season roster and players
    response1 = r.get('https://statsapi.web.nhl.com/api/v1/teams/13/roster/?season=' + season) 
    response2 = r.get('https://statsapi.web.nhl.com/api/v1/people') 
    
    #convert to json dictionary
    response1_1 = response1.json()
    #response1.url
    #further access nested json
    response1_2 = response1_1['roster']
    response1_3 = []
    
    #loop through all players in team for season
    for x in range(persons):
        response1_3.append(response1_2[x]['person'])

    #create data frame from players list
    df = p.DataFrame.from_dict(response1_3)
    #drop columns that are not needed
    df.drop('fullName', axis=1, inplace=True)
    df.drop('link', axis=1, inplace=True)
    #insert corresponding season
    df.insert(1, "Season", str(season), True)
    response1_4 = []
    response1_4 = df.loc[:,'id']

    #empty lists for appending necessary query data
    response2_4 = []
    response2_5 = [] 
    response2_6 = []
    response2_7 = []

    #retrieving player specific information
    for x in range(persons):
        response2 = r.get('https://statsapi.web.nhl.com/api/v1/people' + '/' + str(response1_4[x])) 
        response2_1 = response2.json()
        response2_2 = response2_1['people']
        response2_4.append(response2_2[0]['height'])
        response2_5.append(response2_2[0]['weight'])
        response2_6.append(response2_2[0]['birthDate'])

    #calculating plauer age in season
    for x in range(persons):
        response2_6[x] = int(response2_6[x][0:4])
        response2_7.append(int(season[0:4]) - response2_6[x]) 

    #new columns in dataframe of player info
    df.insert(2, "Age", response2_7, True)
    df.insert(3, "Height", response2_4, True)
    df.insert(4, "Weight", response2_5, True)
    response2_3 = []

    #getting player specific play time for the season
    for x in range(persons):
        response2 = r.get('https://statsapi.web.nhl.com/api/v1/people' + '/' + str(response1_4[x]) + '/stats?stats=statsSingleSeason&season=' + season)
        response2_1 = response2.json()
        #print(response2.url)
        response2_2 = response2_1['stats']
        
        try:
            response2_3.append(response2_2[0]['splits'][0]['stat']['timeOnIce'])
        except:
            print('no stats for ' + str(response1_4[x]) + ' in season: ' + season)
            response2_3.append('0:00')
    
    #new column for time played and returning dataframe
    df.insert(5, "Time_Played", response2_3, True)
    df.rename(columns={'id':'Player_Id'}, inplace=True)
    #df.to_csv(index=False)
    return df

In [1526]:
def all_panthers_player_tables():  
    """
    function that combines data from all Panthers player data from 
    seasons 2014-2015 to 2021-2022
    
    :return: a pandas dataframe.
    """ 
    #player count from  
    #curl  https://statsapi.web.nhl.com/api/v1/teams/13/roster?season=20192020 | grep person -c
    player_data_20142015 = create_panthers_table('20142015', 31)
    player_data_20152016 = create_panthers_table('20152016', 35)
    player_data_20162017 = create_panthers_table('20162017', 34)
    player_data_20172018 = create_panthers_table('20172018', 32)
    player_data_20182019 = create_panthers_table('20182019', 39)
    player_data_20192020 = create_panthers_table('20192020', 30)
    player_data_20202021 = create_panthers_table('20202021', 36)
    player_data_20212022 = create_panthers_table('20212022', 38)

    all_player_data = [player_data_20142015,player_data_20152016,
                       player_data_20162017,player_data_20172018,player_data_20182019,
                       player_data_20192020,player_data_20202021,player_data_20212022 ]

    #final dataframe with all Panthers players info from 2014-2015 season to the 2021-2022 season
    result = p.concat(all_player_data)
    return result

In [1527]:
def create_panthers_game_boxscores_table(season): 
    """
    function that queries the NHL API /schedule and /game endpoints for 
    all Florida Panthers players in respective season.

    :param season: the NHL season in format '20142015'
    :return: a pandas dataframe.
    """ 
    
    #requests to NHL API to obtain schedule and games
    response1 = r.get('https://statsapi.web.nhl.com/api/v1/schedule?season=' + season + '&teamId=13')
    #convert to json dictionary
    response1_1 = response1.json()
    #total games for respective season
    total_games = int(response1_1['totalGames']) - 1
    response1_3 = [ ]

    #getting all games info for the season
    for x in range(total_games):
        response1_2 = response1_1['dates']
        response1_3.append(response1_2[x]['games'][0])

    #create a dataframe from list of games
    df = p.DataFrame.from_dict(response1_3)
    df.rename(columns={'gamePk':'Game_Id'}, inplace=True)

    #calculate if Panthers game is home or away
    home_or_away = []
    for x in range(total_games):
        if(int(response1_3[x]['teams']['away']['team']['id']) == 13):
            home_or_away.append('away')
        else:
            home_or_away.append('home')

    #new column for panthers home or away field
    df.insert(1, "Home_Or_Away", home_or_away, True) 
    int(response1_3[0]['teams']['away']['score'])
    int(response1_3[0]['teams']['home']['score'])

    #calculate if Panthers won or loss game 
    win_loss = []
    for x in range(total_games):
        if(int(response1_3[x]['teams']['away']['score']) >  int(response1_3[x]['teams']['home']['score'])):
            if(home_or_away[x] == 'away'):
                win_loss.append('win') 
            else:
                 win_loss.append('loss') 
        elif(int(response1_3[x]['teams']['away']['score']) <  int(response1_3[x]['teams']['home']['score'])):
            if(home_or_away[x] == 'away'):
                win_loss.append('win') 
            else:
                 win_loss.append('loss') 

    #new column for Panthers win or loss field
    df.insert(2, "Win_Or_Loss", win_loss, True) 
    game_ids = df.loc[:,'Game_Id']

    #retrieve team stats for both team in game
    home_team_stats = []
    away_team_stats = []
    for x in range(total_games):
        response2 = r.get('https://statsapi.web.nhl.com/api/v1/game/' + str(game_ids[x]) +'/boxscore')
        response2_1 = response2.json()
        away_team_stats.append(response2_1['teams']['away']['teamStats'])
        home_team_stats.append(response2_1['teams']['home']['teamStats'])

    #new columns for home and away team stats
    df.insert(3, "Home_Team_Stats", home_team_stats, True) 
    df.insert(4, "Away_Team_Stats", away_team_stats, True) 
    #drop unecessary tables from data frame
    df.drop('link', axis=1, inplace=True)
    df.drop('gameType', axis=1, inplace=True)
    df.drop('season', axis=1, inplace=True)
    df.drop('gameDate', axis=1, inplace=True)
    df.drop('status', axis=1, inplace=True)
    df.drop('teams', axis=1, inplace=True)
    df.drop('venue', axis=1, inplace=True)
    df.drop('content', axis=1, inplace=True)
    #return game data for the season
    return df

In [1528]:
def all_game_boxscores_tables():
    """
    function that combines date from all game boxscores tables from 
    seasons 2014-2015 to 2021-2022
    
    :return: a pandas dataframe.
    """ 
    
    games_data_20142015 = create_panthers_game_boxscores_table('20142015')
    games_data_20152016 = create_panthers_game_boxscores_table('20152016')
    games_data_20162017 = create_panthers_game_boxscores_table('20162017')
    games_data_20172018 = create_panthers_game_boxscores_table('20172018')
    games_data_20182019 = create_panthers_game_boxscores_table('20182019')
    games_data_20192020 = create_panthers_game_boxscores_table('20192020')
    games_data_20202021 = create_panthers_game_boxscores_table('20202021')
    games_data_20212022 = create_panthers_game_boxscores_table('20212022')

    all_game_data = [games_data_20142015,games_data_20152016,games_data_20162017,
                     games_data_20172018,games_data_20182019,games_data_20192020,
                     games_data_20202021,games_data_20212022]

    #final dataframe with all Panthers games data from 2014-2015 season to the 2021-2022 season
    result = p.concat(all_game_data)
    return result

In [1529]:
def create_panthers_player_boxscores_table(season, persons):
    """
     function that queries the NHL API /game endpoint for 
     all Florida Panthers players in respective season.

    :param season: the NHL season in format '20142015'
    :param persons: the number of Panthers players for the season 
    :return: a pandas dataframe.
    """ 
    #requests to NHL API to obtain games
    response3 = r.get('https://statsapi.web.nhl.com/api/v1/schedule?season='+ season +'&teamId=13')
    #convert to json dictionary
    response3_1 = response3.json()
    #total games for respective season
    total_games = int(response1_1['totalGames']) - 1
    response3_3 = [ ]

    #getting all game ids for the season
    for x in range(total_games):
        response3_2 = response3_1['dates']
        response3_3.append(response3_2[x]['games'][0]['gamePk'])
    
    #getting player_ids and respective home_or_away games for game_id
    season_data = create_panthers_game_boxscores_table(season)
    response4_2 = season_data.loc[:,'Home_Or_Away']
    player_data = create_panthers_table(season, persons)
    response4_4 = player_data.loc[:,'Player_Id']
    game_ids = response3_3
    home_or_away = response4_2
    player_ids = response4_4
    
    #lists for players who played in game
    player_ids_y = []
    #list of lists of players who played games in season
    all_player_ids_y = []
    response4_5 = []
    #list of players stats who played in game
    player_y_stats = []
    #list of players stats who played games in season
    all_player_y_stats = []

    #getting Panthers games by game_id for queries
    for x in range(len(game_ids)):
        response4 = r.get('https://statsapi.web.nhl.com/api/v1/game/' + str(game_ids[x]) +'/boxscore')
        response4_1 = response4.json()
         
        #to determine if Panthers are home or away in game
        if(home_or_away[x] == 'away'):
            response4_3 = response4_1['teams']['away']
        else:
            response4_3 = response4_1['teams']['home']

        #to check if Player is in the roster for game
        response4_5 = list(response4_3['players'].keys())
        for y in range(len(response4_5)):
            if(response4_3['players'][response4_5[y]]['person']['rosterStatus'] == 'Y'):
                player_ids_y.append(response4_5[y][2:])

        all_player_ids_y.append(player_ids_y)

        #list of all players in roster for respective game
        for z in range(len(player_ids_y)):
            player_y_stats.append(response4_3['players']['ID' + str(player_ids_y[z])]['stats'])

        all_player_y_stats.append(player_y_stats)

        player_ids_y = []
        player_y_stats = []

    #data frame of game_ids, players in roster, and player stats
    df = p.DataFrame.from_dict(game_ids)
    df.rename(columns={0:'Game_Ids'}, inplace=True)
    df.insert(1, "Player_Ids_In_Roster", all_player_ids_y, True)
    df.insert(2, "Player_Game_Stats",   all_player_y_stats, True)
    return df

In [1530]:
def all_player_boxscores_tables():  
    """
    function that combines data from all Panthers player boxscores 
    tables from seasons 2014-2015 to 2021-2022
    
    :return: a pandas dataframe.
    """ 
    player_boxscores_data_20142015 = create_panthers_player_boxscores_table('20142015', 31)
    player_boxscores_data_20152016 = create_panthers_player_boxscores_table('20152016', 35)
    player_boxscores_data_20162017 = create_panthers_player_boxscores_table('20162017', 34)
    player_boxscores_data_20172018 = create_panthers_player_boxscores_table('20172018', 32)
    player_boxscores_data_20182019 = create_panthers_player_boxscores_table('20182019', 39)
    player_boxscores_data_20212022 = create_panthers_player_boxscores_table('20212022', 38)
    
    all_player_boxscores_data = [ player_boxscores_data_20142015, player_boxscores_data_20152016,
                                player_boxscores_data_20162017, player_boxscores_data_20172018
                               , player_boxscores_data_20182019, player_boxscores_data_20212022 ]

    #final dataframe with all Panthers boxscores players info from 2014-2015 season 
    #to the 2021-2022 season
    result = p.concat(all_player_boxscores_data)
    return result

In [1437]:
#writing the three data frames to local csv files
data1 = all_panthers_player_tables()
data1.to_csv(r'C:\Users\luisq\player_table.csv', 
             encoding='utf-8', index=None, header=True)

data2 = all_game_boxscores_tables()
data2.to_csv(r'C:\Users\luisq\game_boxscores_table.csv', 
             encoding='utf-8', index=None, header=True)

data3 = all_player_boxscores_tables()
data3.to_csv(r'C:\Users\luisq\player_boxscores_table.csv', 
             encoding='utf-8', index=None, header=True)

no stats for 8481442 in season: 20192020


In [1531]:
pip install pandasql

Note: you may need to restart the kernel to use updated packages.


In [1532]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [1535]:
data1.head()

,Player_Id,Season,Age,Height,Weight,Time_Played
0,8448208,20142015,42,"6' 3""",230,1352:54
1,8465185,20142015,37,"6' 3""",210,1430:48
2,8465978,20142015,37,"6' 2""",217,440:52
3,8466285,20142015,35,"5' 10""",192,1903:05
4,8468001,20142015,33,"5' 11""",177,1020:58


In [1536]:
data2.head()

,Game_Id,Home_Or_Away,Win_Or_Loss,Home_Team_Stats,Away_Team_Stats
0,2014010029,home,loss,"{'teamSkaterStats': {'goals': 3, 'pim': 17, 's...","{'teamSkaterStats': {'goals': 3, 'pim': 21, 's..."
1,2014010051,away,win,"{'teamSkaterStats': {'goals': 1, 'pim': 8, 'sh...","{'teamSkaterStats': {'goals': 1, 'pim': 2, 'sh..."
2,2014010065,away,win,"{'teamSkaterStats': {'goals': 5, 'pim': 2, 'sh...","{'teamSkaterStats': {'goals': 4, 'pim': 10, 's..."
3,2014010088,home,loss,"{'teamSkaterStats': {'goals': 0, 'pim': 12, 's...","{'teamSkaterStats': {'goals': 3, 'pim': 12, 's..."
4,2014010100,away,win,"{'teamSkaterStats': {'goals': 4, 'pim': 8, 'sh...","{'teamSkaterStats': {'goals': 1, 'pim': 8, 'sh..."


In [1537]:
data3.head()

,Game_Ids,Player_Ids_In_Roster,Player_Game_Stats
0,2014010029,"[8469638, 8466285, 8477932, 8475755, 8475760, ...","[{}, {}, {}, {'skaterStats': {'timeOnIce': '21..."
1,2014010051,"[8469638, 8466285, 8477932, 8475755, 8475760, ...","[{}, {}, {}, {}, {'skaterStats': {'timeOnIce':..."
2,2014010065,"[8469638, 8466285, 8477932, 8475755, 8475760, ...","[{'skaterStats': {'timeOnIce': '16:08', 'assis..."
3,2014010088,"[8469638, 8466285, 8477932, 8475755, 8475760, ...","[{'skaterStats': {'timeOnIce': '14:11', 'assis..."
4,2014010100,"[8469638, 8466285, 8477932, 8475755, 8475760, ...","[{'skaterStats': {'timeOnIce': '17:52', 'assis..."


In [1538]:
#q1 Success correct query
q = '''SELECT Player_Id, COUNT(*) as Full_Seasons
        FROM data1
        GROUP BY Player_Id
        ORDER BY COUNT(*) DESC
        LIMIT 5
        '''
q1_answer = pysqldf(q)
q1_answer

,Player_Id,Full_Seasons
0,8477932,8
1,8477493,8
2,8476456,8
3,8477346,6
4,8476389,6


In [1549]:
#q2 failed query, instead tried Psuedo Query
#q = '''SELECT Player_Id, Game_Id, plusMinus, Win_Or_Loss 
#       FROM data1 AS player_tables
#       INNER JOIN data1 AS player_boxscore_table ON 
#       player_boxscore_table.Player_Ids_In_Roster = player_tables.Player_Id
#       GROUP BY player_tables.Player_Id
#       ORDER BY plusMinus DESC
#       LIMIT 1
#    '''
#q2_answer = pysqldf(q)
#q2_answer

In [1551]:
#q3 Success except does not return first game_id for each player
q = '''SELECT Player_Id, COUNT(*) as Full_Seasons, Age as First_Season_Age, 
        Season as First_Season
        FROM data1 
        GROUP BY Player_Id
        ORDER BY COUNT(*) DESC
        LIMIT 3
        '''
q3_answer = pysqldf(q)
q3_answer

,Player_Id,Full_Seasons,First_Season_Age,First_Season
0,8477932,8,18,20142015
1,8477493,8,19,20142015
2,8476456,8,21,20142015


In [1550]:
#q4 failed query, instead tried Psuedo Query
#q = '''SELECT Player_Id, MAX(penaltyMinutes), Season,
#        FROM data3 AS player_boxscore_table
#        INNER JOIN data1 AS player_tables ON 
#        player_boxscore_table.Player_Ids_In_Roster = player_tables.Player_Id
#        WHERE penaltyMinutes = MAX(penaltyMinutes)
#        GROUP BY player_tables.Player_Id
#        '''
#q4_answer = pysqldf(q)
#q4_answer



In [ ]:
#q5 failed query, instead tried Psuedo Query
#q = '''SELECT Season, home_shutdown
#        FROM data1 
#        INNER JOIN data2 ON data1.Player_Ids_In_Roster = data2.Player_Id
#        GROUP BY DISTINCT(Season)
#        ORDER BY home_shutdown DESC
#        '''
#q5_answer = pysqldf(q)
#q5_answer